In [15]:
import requests as req
from bs4 import BeautifulSoup as bs
import os
import re

In [2]:
folderPath = 'ebook'
if not os.path.exists(folderPath):
    os.makedirs(folderPath)

In [ ]:
url = 'https://www.gutenberg.org/browse/languages/zh'
res = req.get(url)
soup = bs(res.text,'lxml')
titles = soup.select('li.pgdbetext a')
book = {}
count = 0
for i in titles:
    if i.has_attr('href'):
        link = i['href']
    book = {'title':i.get_text(),'link':f'https://www.gutenberg.org{link}'}
    url = book['link']
    res = req.get(url)
    res.encoding='utf-8'
    soup = bs(res.text,'lxml')
    for i in soup.select('td.unpadded.icon_save a.link'):
        if i.get_text() == 'Plain Text UTF-8':
            book_url = i['href']
    url = f'https://www.gutenberg.org/{book_url}'
    res = req.get(url)
    res.encoding='utf-8'
    soup =bs(res.text,'lxml')
    txt = soup.find('p')
    regex = r'[\u4E00-\u9FFF+《》，、。；]+'
    content = "".join(re.findall(regex,str(txt)))
    print("完成了< " + book['title'] + " >的抓取")
    try:
        with open(f'{folderPath}/{book["title"]}.txt','w',encoding='utf-8')as file:
            file.write(content)
            count += 1
    except (FileNotFoundError,OSError):
        print(book['title'] + '=>這本跳過')
        continue
    if count >= 200:
       break
